In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

The idea here is to fit an optical model of the lenslets to the captured PSF data. The idea is to give background- and noise-free impulse responses. This will be built on the tensorflow core code that runs the PSF optimization.

The problem statement is basically:



$$\arg\min_{\theta_l,\theta_M} \sum_{z=0}^{N_z-1} \left\lVert\left|F^{-1}diag(P_L)F\left[U_z(x,y|\theta_M).*\exp(-j\phi(x,y|\theta_l)\right]\right|^2 - b(x,y|z)\right\lVert$$

where $\theta_l$ represents a parameterized phase mask (i.e. lenslet locations, zernikes), $\theta_M$ represents the parameters of the miniscope build, including mask rotations and tip/tilt. $P_L$ is propagation by distance L (in frequency space), $F$ is DFT, $U_z$ is the wavefront in the pupil due to a point source at plane $z$ in front of the GRIN, and $\phi$ is the phase of the pupil-plane mask. $b(x,y|z)$ is the PSF measured from the as-built system with a point source at distance z-plane $z$.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import miniscope_utils_tf as tf_utils
from miniscope_model import Model as msu_model
import scipy as sc
import scipy.ndimage as ndim
import scipy.misc as misc
from scipy import signal
import scipy.io
from skimage.transform import resize as imresize
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
from IPython import display

import os
from os import listdir
from os.path import isfile, join
import matplotlib.animation as animation

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
zernikes_index = [1,2,3,4]
model = msu_model(target_res=0.004, aberrations = True, zernikes = zernikes_index)  # zsampling options: 'fixed' or 'uniform_random'


In [ ]:
#model = msu_model(target_res=0.004,aberrations = True)  # zsampling options: 'fixed' or 'uniform_random'
load_init_from_file = False
# Load initialization from file
if load_init_from_file == True:
    print('loading initilization from file')
    file_best = '/media/hongdata/Kristina/MiniscopeData/best_init.mat'
    file_worst = '/media/hongdata/Kristina/MiniscopeData/worst_init.mat'
    model = load_model_from_file(model, file_best)
    
# Save initial values for later comparison 
Rmat=model(0)
R_init = Rmat
Tinit,_,_= tf_utils.make_lenslet_tf_zern(model)

xinit = tfe.Variable(tf.zeros(model.Nlenslets));       tf.assign(xinit, model.xpos)
yinit = tfe.Variable(tf.zeros(model.Nlenslets));       tf.assign(yinit, model.ypos)
rinit = tfe.Variable(tf.zeros(model.Nlenslets));       tf.assign(rinit, model.rlist)
#offsetinit = tfe.Variable(tf.zeros(model.Nlenslets));  tf.assign(offsetinit, model.lenslet_offset)
zerninit = tfe.Variable(tf.zeros((model.Nlenslets, model.numzern))); tf.assign(zerninit, model.zernlist)

model_init = msu_model(target_res=0.004, aberrations = True, zernikes = zernikes_index)
tf.assign(model_init.xpos, xinit)
tf.assign(model_init.ypos, yinit)
tf.assign(model_init.rlist, rinit)
#tf.assign(model_init.lenslet_offset, offsetinit)
tf.assign(model_init.zernlist, zerninit)
Rmat_init = model_init(0)

fig=plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.imshow(Tinit.numpy())
plt.subplot(1,2,2)
plt.plot(R_init.numpy())

In [ ]:
zstack = [tf.ones((5,5)) for n in range(6)]
model.target_psf = [.5*tf.ones((5,5)) for n in range(6)]
testloss = [tf.norm(test1[n]-test2[n]) for n in range(6)]

In [ ]:

[0.5*tf.norm(zstack[n] - model.target_psf[n])**2 for n in range(6)]

In [ ]:
psf_zstack =  model.gen_psf_stack(Tinit,.9,0)
psf_zstack_lst = [np.array(psf_zstack[n].numpy()) for n in range(model.Nz)]
psf_zstack_arr = np.array(psf_zstack_lst)
os.mkdir("../psf_meas/")

In [ ]:
sc.io.savemat("../psf_meas/zstack_sim_test.mat",{'zstack':psf_zstack_arr})

In [ ]:
test_psf_in = sc.io.loadmat("../psf_meas/zstack_sim_test.mat")

In [ ]:
print(np.shape(test_psf_in['zstack']))
test_psf_tf = [tf.constant(test_psf_in['zstack'][n]) for n in range(np.shape(test_psf_in['zstack'])[0])]
print(test_psf_tf)